# Clustering Crypto

In [62]:
# Initial imports
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path


### Fetching Cryptocurrency Data

In [63]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [64]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [65]:
# Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df.head(20)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [66]:
crypto_df.describe()
# Create a DataFrame

,TotalCoinsMined
count,7.440000e+02
mean,3.883103e+09
std,3.867263e+10
min,-5.917978e+09
25%,6.322164e+05
50%,1.340427e+07
75%,9.399895e+07
max,9.899887e+11


### Data Preprocessing

In [67]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df_cleaned = crypto_df.set_index("Unnamed: 0")
crypto_df_cleaned.index.names = ['']
crypto_df_cleaned

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000


In [68]:
# Keep only cryptocurrencies that are trading
crypto_df_cleaned = crypto_df_cleaned[crypto_df_cleaned.IsTrading == True]
crypto_df_cleaned

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610


In [69]:
# Remove the "IsTrading" column
crypto_df_cleaned = crypto_df_cleaned.drop(columns = "IsTrading")
crypto_df_cleaned

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [70]:
# Remove rows with at least 1 null value
crypto_df_cleaned = crypto_df_cleaned.dropna()
crypto_df_cleaned

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [71]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df_cleaned = crypto_df_cleaned[crypto_df_cleaned.TotalCoinsMined > 0]
crypto_df_cleaned

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [72]:
# Drop rows where there are 'N/A' text values
for column in crypto_df_cleaned.columns:
    print(f"Column {column} has {(crypto_df_cleaned[column]=='N/A').sum()} N/A values")

Column CoinName has 0 N/A values
Column Algorithm has 0 N/A values
Column ProofType has 0 N/A values
Column TotalCoinsMined has 0 N/A values
Column TotalCoinSupply has 0 N/A values


In [73]:
for column in crypto_df_cleaned.columns:
    print(f"Column {column} has {(crypto_df_cleaned[column]=='N/A').sum()} N/A values")

Column CoinName has 0 N/A values
Column Algorithm has 0 N/A values
Column ProofType has 0 N/A values
Column TotalCoinsMined has 0 N/A values
Column TotalCoinSupply has 0 N/A values


In [74]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = pd.DataFrame(crypto_df_cleaned['CoinName'], index = crypto_df_cleaned.index)
coin_names

,CoinName
,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex


In [75]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df_cleaned = crypto_df_cleaned.drop(columns = 'CoinName')
crypto_df_cleaned

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [76]:
# Create dummy variables for text features
X_binary_encoded = pd.get_dummies(crypto_df_cleaned, columns=['Algorithm', 'ProofType'])
X_binary_encoded

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_binary_encoded)

### Reducing Dimensions Using PCA

In [78]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [79]:
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.34887031,  0.93455864, -0.55413621],
       [-0.33217558,  0.93477594, -0.55446611],
       [ 2.31630654,  1.69192699, -0.65068911],
       ...,
       [ 0.33752154, -2.20984551,  0.43414198],
       [-0.1396541 , -1.98223801,  0.44573733],
       [-0.28467156,  0.85942906, -0.28719433]])

In [80]:
# Create a DataFrame with the principal components data
pca_data = pd.DataFrame(data=X_pca, columns=["PC1", "PC2", "PC3"], index = crypto_df_cleaned.index)
pca_data.head(10)

,PC1,PC2,PC3
,,,
42,-0.348870,0.934559,-0.554136
404,-0.332176,0.934776,-0.554466
1337,2.316307,1.691927,-0.650689
BTC,-0.127278,-1.277751,0.184679
ETH,-0.144845,-1.998873,0.400400
LTC,-0.168042,-1.148142,0.005784
DASH,-0.407951,1.158137,-0.504665
XMR,-0.139688,-2.154554,0.416490
ETC,-0.143284,-1.998957,0.400384


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [81]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_data)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_curve = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_curve)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [82]:
# Initialize the K-Means model
kmeans_model = KMeans(n_clusters=4, random_state=0)
# Fit the model
kmeans_model.fit(pca_data)
# Predict clusters
cluster_prediction = kmeans_model.predict(pca_data)

In [83]:
# Creating a new DataFrame including predicted clusters and cryptocurrencies features
new_df = pd.DataFrame(kmeans_model.labels_, index = crypto_df_cleaned.index)
new_df.columns = ["Class"]
combined_df = pd.concat([crypto_df_cleaned, pca_data, coin_names, new_df], axis = "columns", join = "inner")
combined_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.348870,0.934559,-0.554136,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.332176,0.934776,-0.554466,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.316307,1.691927,-0.650689,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.127278,-1.277751,0.184679,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.144845,-1.998873,0.400400,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168042,-1.148142,0.005784,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.407951,1.158137,-0.504665,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.139688,-2.154554,0.416490,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.143284,-1.998957,0.400384,Ethereum Classic,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [84]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    combined_df,
    x="PC3",
    y="PC2",
    z="PC1",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [85]:
# Table with tradable cryptos
combined_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [86]:
# Print the total number of tradable cryptocurrencies
print(f"There are {combined_df.shape[0]} tradable crytocurrencies in our dataset.")


There are 532 tradable crytocurrencies in our dataset.


#### Scatter Plot with Tradable Cryptocurrencies

In [87]:
# Scale data to create the scatter plot
tradable_scatter = combined_df.copy()
tradable_scatter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532 entries, 42 to XBC
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        532 non-null    object 
 1   ProofType        532 non-null    object 
 2   TotalCoinsMined  532 non-null    float64
 3   TotalCoinSupply  532 non-null    object 
 4   PC1              532 non-null    float64
 5   PC2              532 non-null    float64
 6   PC3              532 non-null    float64
 7   CoinName         532 non-null    object 
 8   Class            532 non-null    int32  
dtypes: float64(4), int32(1), object(4)
memory usage: 39.5+ KB


In [98]:
tradable_scatter.TotalCoinSupply = tradable_scatter.TotalCoinSupply.astype(np.float64)

tradable_scatter = tradable_scatter[tradable_scatter.TotalCoinSupply > 0]
tradable_scatter

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class,TotalCoinsMinedLog
,,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.348870,0.934559,-0.554136,42 Coin,0,3.737669
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.332176,0.934776,-0.554466,404Coin,0,20.776982
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.316307,1.691927,-0.650689,EliteCoin,0,24.100151
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.127278,-1.277751,0.184679,Bitcoin,1,16.701828
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.168042,-1.148142,0.005784,Litecoin,1,17.959268
...,...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2.000000e+09,2.452271,0.700382,0.047944,ZEPHYR,0,21.416413
GAP,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08,-0.346912,0.934462,-0.554157,Gapcoin,0,16.518953
BDX,CryptoNight,PoW,9.802226e+08,1.400223e+09,0.337522,-2.209846,0.434142,Beldex,1,20.703290


In [99]:
#in logarithmic scale
tradable_scatter['TotalCoinsMinedLog'] = np.log(tradable_scatter['TotalCoinsMined'])
tradable_scatter

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class,TotalCoinsMinedLog
,,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.348870,0.934559,-0.554136,42 Coin,0,3.737669
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.332176,0.934776,-0.554466,404Coin,0,20.776982
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.316307,1.691927,-0.650689,EliteCoin,0,24.100151
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.127278,-1.277751,0.184679,Bitcoin,1,16.701828
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.168042,-1.148142,0.005784,Litecoin,1,17.959268
...,...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2.000000e+09,2.452271,0.700382,0.047944,ZEPHYR,0,21.416413
GAP,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08,-0.346912,0.934462,-0.554157,Gapcoin,0,16.518953
BDX,CryptoNight,PoW,9.802226e+08,1.400223e+09,0.337522,-2.209846,0.434142,Beldex,1,20.703290


In [100]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot = px.scatter(
    tradable_scatter,
    title="Tradable Cryptocurrency Scatter Plot",
    x="TotalCoinsMinedLog", 
    y="TotalCoinSupply", 
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    width=800,
)
plot.update_layout(legend=dict(x=0, y=1))
plot.show()